<img src="https://duckdb.org/images/DuckDB-Footer.svg" style="width:8em">

# hvPlot + DuckDB for fast interactive analytics

By combining hvPlot and [DuckDB](https://duckdb.org/) you can create very fast interactive analytics solutions.

hvPlot integrates with DuckDB via [Ibis](https://ibis-project.org/docs/latest/). You can install the combination via `pip install ibis-framework[duckdb]`.

<div class="alert alert-warning">
Note: The support for Ibis is experimental and you might experience rough edges. Please report any issues and feedback at <a href="https://github.com/holoviz/hvplot">github/hvplot</a>
</div>

In [ ]:
import duckdb
import ibis
import pandas as pd

from pathlib import Path

Lets start by creating a DuckDB database

In [ ]:
DUCKDB_PATH = "example_duckdb.db"

if not Path(DUCKDB_PATH).exists():
    pandas_df = pd.DataFrame(
        {
            "actual": [100, 150, 125, 140, 145, 135, 123],
            "forecast": [90, 160, 125, 150, 141, 141, 120],
            "numerical": [1.1, 1.9, 3.2, 3.8, 4.3, 5.0, 5.5],
            "date": pd.date_range("2022-01-03", "2022-01-09"),
            "string": ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"],
        },
    )
    duckdb_con = duckdb.connect(DUCKDB_PATH)
    duckdb_con.execute("CREATE TABLE df AS SELECT * FROM pandas_df")

Before starting you will have have to specify the amount of data you will be working on. By default Ibis work on `10000` rows of data whenever its `.execute` method is run.

Here we change that setting to `None` to use all the data in the tables. But using a limited number of rows can be useful for the initial exploration and development.

In [ ]:
ibis.options.sql.default_limit = None

## Connect to duckDB

Lets create the connection to the DuckDB database

In [ ]:
db = ibis.duckdb.connect(DUCKDB_PATH)

And the table

In [ ]:
table = db.table("df")

Let us check the data

In [ ]:
table.execute()

and schema

In [ ]:
table.schema()

## Basic Plots

To use hvPlot with the ibis-duckdb data source you will need to `import hvplot.ibis`. This will add the `.hvplot` method to the `table` above.

In [ ]:
import hvplot.ibis

Let us make a plots

In [ ]:
COLOR_MAP = ["#4E79A7", "#F28E2B", "#E15759"]

table.hvplot(x="numerical", y=["actual", "forecast"], line_width=5, color=COLOR_MAP)

Let us make some more

In [ ]:
table.hvplot.bar(x="string", y="actual", line_width=2, color=COLOR_MAP[0], alpha=0.7) *\
table.hvplot.line(x="string", y="forecast", line_width=5, color=COLOR_MAP[2])

## Big Data Viz with DataShader